In [ ]:
#This code logs onto the HCP S3 bucket and searches for X participants with 7t data, then downloads the T1, T1 mask, and dwi mask for use in template generation
#To generate a template of this data, run the following line of code using mrtrix3/3.0.2 in terminal
#population_template 'template_output.mif' -warp_dir warps -voxel_size 0.7 -mask_dir 'mask_output_folder_directory -force
#The warps output is kept for each T1 as this is useful data for transforming further T1s/segmentations to confirm to the 184 template

In [ ]:
import boto3
import os

# Sign up for an account at https://db.humanconnectome.org/. 
# Here you will be given an Amazon S3 access key/secret access code
# Input these below

# Initialize the S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id='',
    aws_secret_access_key=''
)
bucket_name = 'hcp-openaccess'
download_dir = '' #define where you want the downloads to go
prefix = 'HCP_1200/'

download_count = 0

print("Fetching list of folders from S3...")

response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter='/')
if 'CommonPrefixes' in response:
    # Extract folder names (6-digit numbers)
    folders = [folder['Prefix'].split('/')[1] for folder in response['CommonPrefixes']]
    for folder in folders:
        # Construct the S3 key to check if the Diffusion_7T folder exists inside T1w
        diffusion_folder_key = f'HCP_1200/{folder}/T1w/Diffusion_7T/data.nii.gz'
        
        try:
            s3.head_object(Bucket=bucket_name, Key=diffusion_folder_key)
            print(f"Found Diffusion_7T in folder {folder}.")

            # Make keys for the T1 image
            s3_key_T1 = f'HCP_1200/{folder}/T1w/T1w_acpc_dc.nii.gz'
            s3_key_mask = f'HCP_1200/{folder}/T1w/brainmask_fs.nii.gz'
            s3_key_dwi_mask = f'HCP_1200/{folder}/T1w/Diffusion_7T/nodif_brain_mask.nii.gz'
            
            # Define the local file path with the folder name
            local_file_name = f"{folder}_T1.nii.gz"
            local_file_name_T1w_mask =f"{folder}_T1w_mask.nii.gz"
            local_file_name_dwi_mask =f"{folder}_dwi_mask.nii.gz"
            
            local_file_path_T1w = os.path.join(download_dir+'/T1w', local_file_name)
            local_file_path_T1w_mask = os.path.join(download_dir+'/T1w_masks', local_file_name_T1w_mask)
            local_file_path_dwi_mask = os.path.join(download_dir+'/dwi_masks', local_file_name_dwi_mask)
            
            # Download the file
            print(f"Downloading {folder} T1.")
            s3.download_file(bucket_name, s3_key_T1, local_file_path_T1w)
            print(f"Downloading {folder} T1w Mask.")
            s3.download_file(bucket_name, s3_key_mask, local_file_path_T1w_mask)
            print(f"Downloading {folder} DWI mask.")
            s3.download_file(bucket_name, s3_key_dwi_mask, local_file_path_dwi_mask)
                
            # Increment the download count
            download_count += 1
            print(f"Downloaded {download_count} files.")

            if download_count >= 184:  # Change to 40 or the desired number
                break

        except s3.exceptions.ClientError:
            # If Diffusion_7T folder is not found, skip this folder
            print(f"Diffusion_7T folder not found in folder {diffusion_folder_key}. Skipping...")

else:
    print("No folders found in the S3 bucket with the specified prefix.")

print("Finished downloading files.")